In [61]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import datetime as dt
import scipy.optimize as sco
from util import get_data, plot_data

In [45]:
def assess_portfolio(allocs,prices):
    prices_normed = prices/prices.values[0]
    alloced = prices_normed * allocs
    port_val = alloced.sum(axis=1)
    daily_retns = port_val[1:].values/port_val[:-1] - 1
    
    # cumulative return
    cr = daily_retns[-1]/daily_retns[0]-1
    
    # average daily return
    adr = daily_retns.mean()
    
    #standard deviation of daily return
    sddr = daily_retns.std()
    
    # sharpe ratio, risk adjusted (0%) return
    sr = (adr - 0) / sddr
    sr *= 252**0.5
    
    return port_val, cr, adr, sddr, sr

In [53]:
def min_sharpe_ratio(allocs,prices):
    return - assess_portfolio(allocs,prices)[4]

In [55]:
def optimize_portfolio(
    sd=dt.datetime(2008,1,1),
    ed=dt.datetime(2009,1,1),
    syms=['GOOG','AAPL','GLD','XOM'],
    gen_plot=False,
):
    dates=pd.date_range(sd,ed)
    prices_all=get_data(syms,dates)
    prices=prices_all[syms]
    prices_SPY=prices_all['SPY']
    
    # find the allocations for the optimal portfolio
    # note that the values here ARE NOT meant to be correct for a test case
    num_stocks = len(syms)
    initial_alloc = [1/num_stocks]*num_stocks
    bds = [(0.0,1.0)] * num_stocks
    cons = ({'type':'eq','fun': lambda x : np.sum(x)-1})
    
    opts = sco(min_sharpe_ratio, initial_alloc, method='SLSQP',bounds=bds, constraints=cons).x
    
    port_val, cr, adr, sddr, sr = assess_portfolio(opts, prices)
    
    
    if gen_plot:
        df_temp = pd.concat([port_val / port_val[0,:], prices_SPY / prices_SPY[0,:]], keys=['Portfolio', 'SPY'], axis=1)
        ax = df_temp.plot()
        ax.set_title('Daily Portfolio Value and SPY')
        ax.set_ylabel('Price')
        ax.set_xlabel('Date')
        plt.grid(True)
        plt.show()
    
    return opts,cr,adr,sddr,sr

In [58]:
def test_code():
    start_date = dt.datetime(2009,1,1)
    end_date = dt.datetime(2010,1,1)
    symbols=['GOOG','AAPL','GLD','XOM','IBM']
    
    # Assess the portfolio
    allocations, cr,adr,sddr,sr=optimize_portfolio(
        sd=start_date, ed=end_date, syms=symbols, gen_plot=False
    )
    
    # Print statistics
    print('Start Date:', start_date)
    print('End Date:', end_date)
    print('Symbols:', symbols)
    print('Allocations:', allocations)
    print('Sharpe Ratio:', sr)
    print('Volatility (stdev of daily returns):', sddr)
    print('Average Daily Return:', adr)
    print('Cumulative Return:', cr)

In [60]:
if __name__ == 'main':
    test_code()